In [45]:
import requests
from http.server import BaseHTTPRequestHandler, HTTPServer
import webbrowser

#Source
#https://cwsi.pl/ecommerce/allegro/allegro-pl-rest-api-w-pythonie-wprowadzenie/
#Improvements:
## Removed api key

## TODO 

# Dla ułatwienia, definiujemy domyślne wartości (tak zwane stałe), są one uniwersalne
DEFAULT_OAUTH_URL = 'https://allegro.pl/auth/oauth'
DEFAULT_REDIRECT_URI = 'http://localhost:8000'

# Implementujemy funkcję, której parametry przyjmują kolejno:
#  - client_id (ClientID), api_key (API Key) oraz opcjonalnie redirect_uri i oauth_url
# (jeżeli ich nie podamy, zostaną użyte domyślne zdefiniowane wyżej)
def get_access_code(client_id, api_key, redirect_uri=DEFAULT_REDIRECT_URI, oauth_url=DEFAULT_OAUTH_URL):
    # zmienna auth_url zawierać będzie zbudowany na podstawie podanych parametrów URL do zdobycia kodu
    auth_url = '{}/authorize' \
               '?response_type=code' \
               '&client_id={}' \
               '&api-key={}' \
               '&redirect_uri={}'.format(oauth_url, client_id, api_key, redirect_uri)
 
    # uzywamy narzędzia z modułu requests - urlparse - służy do spardowania podanego url 
    # (oddzieli hostname od portu)
    parsed_redirect_uri = requests.utils.urlparse(redirect_uri)

    # definiujemy nasz serwer - który obsłuży odpowiedź allegro (redirect_uri)
    server_address = parsed_redirect_uri.hostname, parsed_redirect_uri.port

    # Ta klasa pomoże obsłużyć zdarzenie GET na naszym lokalnym serwerze
    # - odbierze żądanie (odpowiedź) z serwisu allegro
    class AllegroAuthHandler(BaseHTTPRequestHandler):
        def __init__(self, request, address, server):
            super().__init__(request, address, server)

        def do_GET(self):
            self.send_response(200, 'OK')
            self.send_header('Content-Type', 'text/html')
            self.end_headers()

            self.server.path = self.path
            self.server.access_code = self.path.rsplit('?code=', 1)[-1]

    # Wyświetli nam adres uruchomionego lokalnego serwera
    print('server_address:', server_address)

    # Uruchamiamy przeglądarkę, przechodząc na adres zdefiniowany do uzyskania kodu dostępu
    # wyświetlić się powinien formularz logowania do serwisu Allegro.pl
    webbrowser.open(auth_url)

    # Uruchamiamy nasz lokalny web server na maszynie na której uruchomiony zostanie skrypt
    # taki serwer dostępny będzie pod adresem http://localhost:8000 (server_address)
    httpd = HTTPServer(server_address, AllegroAuthHandler)
    print('Waiting for response with access_code from Allegro.pl (user authorization in progress)...')

    # Oczekujemy tylko jednego żądania
    httpd.handle_request()

    # Po jego otrzymaniu zamykamy nasz serwer (nie obsługujemy już żadnych żądań)
    httpd.server_close()

    # Klasa HTTPServer przechowuje teraz nasz access_code - wyciągamy go
    _access_code = httpd.access_code

    # Dla jasności co się dzieje - wyświetlamy go na ekranie
    print('Got an authorize code: ', _access_code)

    # i zwracamy jako rezultat działania naszej funkcji
    return _access_code


#Generating token which you can load 
def sign_in(client_id, client_secret, access_code, redirect_uri=DEFAULT_REDIRECT_URI, oauth_url=DEFAULT_OAUTH_URL):
    token_url = oauth_url + '/token'

    access_token_data = {'grant_type': 'authorization_code',
                         'code': access_code,
                         'redirect_uri': redirect_uri}

    response = requests.post(url=token_url,
                             auth=requests.auth.HTTPBasicAuth(client_id, client_secret),
                             data=access_token_data)

    return response.json()


#Each 3 hours you have to refresh your token
def post_refresh_token(client_id, client_secret, refresh_token, redirect_uri=DEFAULT_REDIRECT_URI, oauth_url=DEFAULT_OAUTH_URL):
    token_url = oauth_url + '/token'

    access_token_data = {'grant_type': 'refresh_token',
                         'refresh_token': refresh_token,
                         'redirect_uri': redirect_uri}

    response = requests.post(url=token_url,
                             auth=requests.auth.HTTPBasicAuth(client_id, client_secret),
                             data=access_token_data)

    return response.json()

In [46]:
import fbchat 
from getpass import getpass 
from fbchat import Client
from fbchat.models import *
import pandas as pd
from pandas.io.json import json_normalize
import urllib.request, json
import datetime as dt
from pymongo import MongoClient
import time
import re
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

f = open('config/config.json') 

config_json = json.load(f) 
  
f.close() 

In [47]:
#Allegro config
client_id     = config_json['allegro']['client_id']
client_secret = config_json['allegro']['client_secret']

access_code   = get_access_code(client_id,client_secret)

access_tokens = sign_in(client_id, client_secret, access_code)

server_address: ('localhost', 8000)
Waiting for response with access_code from Allegro.pl (user authorization in progress)...


127.0.0.1 - - [23/Oct/2020 14:27:26] "GET /?code=QBh35SGeel6MwwTDIjnwm6vnaiOO89gm HTTP/1.1" 200 -


Got an authorize code:  QBh35SGeel6MwwTDIjnwm6vnaiOO89gm


In [49]:
access_token  = access_tokens['access_token']
refresh_token = access_tokens['refresh_token']

In [50]:
DEFAULT_API_URL = 'https://api.allegro.pl'
headers = {}
headers['charset'] = 'utf-8'
headers['Accept-Language'] = 'pl-PL'
headers['Content-Type'] = 'application/json'
#headers['Api-Key'] = api_key
headers['Accept'] = 'application/vnd.allegro.public.v1+json'
headers['Authorization'] = "Bearer {}".format(access_token)


In [51]:
headers = {}
headers['charset'] = 'utf-8'
headers['Accept-Language'] = 'pl-PL'
headers['Content-Type'] = 'application/json'
headers['Accept'] = 'application/vnd.allegro.public.v1+json'
headers['Authorization'] = "Bearer {}".format(access_token)

#List all auctions (limit - 60):
def searchAllegro(keyword, used_only=False,default_api_url = 'https://api.allegro.pl'):
    with requests.Session() as session:
        session.headers.update(headers)
        url = default_api_url + '/offers/listing?phrase='+keyword+'&sort=-startTime'
        if used_only: 
            url = url + "&parameter.11323=11323_2"
            
        response = session.get(url)

        return(response.json())

In [52]:
def categoryAllegro(category,default_api_url = 'https://api.allegro.pl'):
    url= '/sale/categories/' + category + ''
    with requests.Session() as session:
        session.headers.update(headers)
        response = session.get(default_api_url + url)
    print(url)
    return(response.json())

In [53]:
def categoryAggAllegro(category):
    i=1
    deepers={}
    deepers[i]=categoryAllegro(category)
    #deepers[i]['path']= deepers[i]['name']
    while True:
        deepers[i+1] = categoryAllegro(deepers[i]['parent']['id'])

        i=i+1
        if deepers[i]['parent'] is None:
            break
    lenght_deepers=len(deepers)
    deepers[lenght_deepers]['path']= deepers[lenght_deepers]['name']

    for i in range(lenght_deepers-1,0,-1):
        deepers[i]['path']= deepers[i]['name']+ '/' + deepers[i+1]['path']

    return deepers

In [54]:
import fbchat 
from getpass import getpass 
from fbchat import Client
from fbchat.models import *
username = config_json['messenger']['username']
password = config_json['messenger']['password']

#client = fbchat.Client(username,password,max_tries=1)

#Test message:
#client.send(Message(text='https://allegro.pl/oferta/9792024126'), thread_id=client.uid)

In [56]:
#https://github.com/fbchat-dev/fbchat/issues/615
user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko'

import logging
#logging.basicConfig(level=logging.DEBUG)

client = fbchat.Client(username,password,user_agent=user_agent,max_tries=1, logging_level=logging.DEBUG) 


Logging in jacoszekm@gmail.com...


Please enter your 2FA code -->  224659


Submitting 2FA code.
Saving browser 2.
Saving browser1.
Starting Facebook checkup flow.
Verifying login attempt.
Saving device again.
tu cos nie ak
fbgt
path1.
Login of jacoszekm@gmail.com successful.


In [57]:
#client = fbchat.Client(username,password) 

In [58]:
#user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko'
#client = fbchat.Client(username,password,user_agent=user_agent) 

#cookies = {}
#try:
#    with open('config/fb_session.json', 'r') as f:
#        cookies = json.load(f)
#except:
#    pass
#
#client = fbchat.Client(username, password, session_cookies=cookies)

#with open('config/fb_session.json', 'w') as f:
#    json.dump(client.getSession(), f)

In [59]:
mongo_connection_string=config_json['mongodb']['connection_string']

In [60]:
with MongoClient(mongo_connection_string) as client_mongo:
    mydb= client_mongo["allegro"]
    search_tbl = mydb["searches"]
    search_list=list(search_tbl.find({}))
    search_list=json_normalize(search_list)
    #search_list=[d['name'] for d in search_list]

In [ ]:
with MongoClient(mongo_connection_string) as client_mongo:
    mydb= client_mongo["allegro"]
    search_tbl = mydb["searches"]
    search_list=list(search_tbl.find({}))
    search_list=json_normalize(search_list)

i = 10
new_rows=1
while True: 
    
    #Select all ids which you have
    if new_rows==1:
        with MongoClient(mongo_connection_string) as client_mongo:
            mydb= client_mongo["allegro"]
            auction_tbl = mydb["auctions"]
            auction_list=list(auction_tbl.find({},{'id':1, "_id": 0}))
            auction_list=[d['id'] for d in auction_list]
            
        with MongoClient(mongo_connection_string) as client_mongo:
            mydb= client_mongo["allegro"]
            auction_tbl = mydb["categories"]
            category_dict=list(auction_tbl.find({}))
            category_list=[d['id'] for d in category_dict]
            category_table=json_normalize(category_dict)

        new_rows = 0 
        
    #Iterate over keywords
    #for k in keywords:
    for index, row in search_list.iterrows():
        print(row['keyword'])
        k=row['keyword']
        
        used_only=False
        if row['used_only']==True: 
            used_only=True
            
        search_result=searchAllegro(k,used_only)

        table = json_normalize(search_result['items']['promoted'] + search_result['items']['regular'])
        table = table.rename(columns=lambda x: re.sub('\\.','',x))
        current_time = dt.datetime.now()
        table['keyword'] = k
        table['record_creation_date']=current_time.strftime('%Y-%m-%d %H:%M:%S')

        if 'id' in table:
            print('has')
            to_add=table[~table['id'].isin(auction_list)]

            #Update categories
            categories_to_add = np.setdiff1d(to_add['categoryid'].unique().tolist(),category_list).tolist()

            #for c in categories_to_add:
            #    with MongoClient(mongo_connection_string) as client_mongo:
            #        mydb     = client_mongo["allegro"]
            #        mycol    = mydb["categories"]
            #        rows_json=categoryAllegro(c)
            #        mycol.insert_one(rows_json)

            for c in categories_to_add:
                if c != '77925':
                    with MongoClient(mongo_connection_string) as client_mongo:
                        mydb     = client_mongo["allegro"]
                        mycol    = mydb["categories"]
                        rows_json=categoryAggAllegro(c)
                        for key, value in rows_json.items():
                            mycol.insert_one(value)

            #UPDATE DICTIONARY
            if len(categories_to_add) > 0:    
                with MongoClient(mongo_connection_string) as client_mongo:
                    mydb= client_mongo["allegro"]
                    auction_tbl = mydb["categories"]
                    category_dict=list(auction_tbl.find({}))
                    category_list=[d['id'] for d in category_dict]
                    category_table=json_normalize(category_dict)

            to_add_size = to_add.shape[0]
            
            if to_add_size>0:
                print('adding rows')
                new_rows = 1
                #If we have few auctions just present them:
                if to_add_size <= 5:
                    for index, row in to_add.iterrows():
                        
                        category_desc=''
                        if  row['categoryid'] != '77925':
                            category_desc=category_table.path[category_table['id']==row['categoryid']].values[0] + '\n'
                            
                        text_message= category_desc + row['name'] + ' - ' + row['sellingModeformat'] + ' - ' + row['sellingModepriceamount'] + '\n'+ 'https://allegro.pl/oferta/' + row['id']
                        client.send(Message(text=text_message), thread_id=client.uid)
                        print(text_message) 
                        
               #Overcomining fb messsneger limits:
                else:
                    text_message=''

                    for index, row in to_add.iterrows():
                        text_message=text_message+row['name'] + ' - ' + row['sellingModeformat'] + ' - '  + row['sellingModepriceamount'] + '\n'+ 'https://allegro.pl/oferta/' + row['id']+ '\n'+ '\n'

                    print(text_message) 
                    client.send(Message(text=text_message), thread_id=client.uid)

                with MongoClient(mongo_connection_string) as client_mongo:
                    mydb     = client_mongo["allegro"]
                    auction_tbl = mydb["auctions"]
                    rows_json=to_add.to_dict(orient='records')
                    auction_tbl.insert_many(rows_json)
                    #print('added')

        #Po 10 pętlach odśwież token (3h ważny)
    if(i == 10):
        refreshed_token=post_refresh_token(client_id, client_secret, refresh_token) 
        i=0
        
        access_token  = refreshed_token['access_token']
        refresh_token = refreshed_token['refresh_token']
        headers['Authorization'] = "Bearer {}".format(access_token)
        #print("refreshed")
        
    i=i+1
    #print(i)
    time.sleep(600)

dreamcast
has
atari%20-st%20-2600%20-7800
has
pamiętniki%20chłopów
has
knex
has
honda accord IV -bezpiecznik -kołpaki
has


In [ ]:
with MongoClient(mongo_connection_string) as client_mongo:
    mydb= client_mongo["allegro"]
    search_tbl = mydb["searches"]
    search_list=list(search_tbl.find({}))
    search_list=json_normalize(search_list)

i = 10
new_rows=1
while True: 
    
    #Select all ids which you have
    if new_rows==1:
        with MongoClient(mongo_connection_string) as client_mongo:
            mydb= client_mongo["allegro"]
            auction_tbl = mydb["auctions"]
            auction_list=list(auction_tbl.find({},{'id':1, "_id": 0}))
            auction_list=[d['id'] for d in auction_list]
            
        with MongoClient(mongo_connection_string) as client_mongo:
            mydb= client_mongo["allegro"]
            auction_tbl = mydb["categories"]
            category_dict=list(auction_tbl.find({}))
            category_list=[d['id'] for d in category_dict]
            category_table=json_normalize(category_dict)

        new_rows = 0 
        
    #Iterate over keywords
    #for k in keywords:
    for index, row in search_list.iterrows():
        print(row['keyword'])
        k=row['keyword']
        
        used_only=False
        if row['used_only']==True: 
            used_only=True
            
        search_result=searchAllegro(k,used_only)

        table = json_normalize(search_result['items']['promoted'] + search_result['items']['regular'])
        table = table.rename(columns=lambda x: re.sub('\\.','',x))
        current_time = dt.datetime.now()
        table['keyword'] = k
        table['record_creation_date']=current_time.strftime('%Y-%m-%d %H:%M:%S')

        if 'id' in table:
            print('has')
            to_add=table[~table['id'].isin(auction_list)]

            #Update categories
            categories_to_add = np.setdiff1d(to_add['categoryid'].unique().tolist(),category_list).tolist()

            #for c in categories_to_add:
            #    with MongoClient(mongo_connection_string) as client_mongo:
            #        mydb     = client_mongo["allegro"]
            #        mycol    = mydb["categories"]
            #        rows_json=categoryAllegro(c)
            #        mycol.insert_one(rows_json)

            for c in categories_to_add:
                if c != '77925':
                    with MongoClient(mongo_connection_string) as client_mongo:
                        mydb     = client_mongo["allegro"]
                        mycol    = mydb["categories"]
                        rows_json=categoryAggAllegro(c)
                        for key, value in rows_json.items():
                            mycol.insert_one(value)

            #UPDATE DICTIONARY
            if len(categories_to_add) > 0:    
                with MongoClient(mongo_connection_string) as client_mongo:
                    mydb= client_mongo["allegro"]
                    auction_tbl = mydb["categories"]
                    category_dict=list(auction_tbl.find({}))
                    category_list=[d['id'] for d in category_dict]
                    category_table=json_normalize(category_dict)

            to_add_size = to_add.shape[0]
            
            if to_add_size>0:
                print('adding rows')
                new_rows = 1
                #If we have few auctions just present them:
                if to_add_size <= 5:
                    for index, row in to_add.iterrows():
                        
                        category_desc=''
                        if  row['categoryid'] != '77925':
                            category_desc=category_table.path[category_table['id']==row['categoryid']].values[0] + '\n'
                            
                        text_message= category_desc + row['name'] + ' - ' + row['sellingModeformat'] + ' - ' + row['sellingModepriceamount'] + '\n'+ 'https://allegro.pl/oferta/' + row['id']
                        client.send(Message(text=text_message), thread_id=client.uid)
                        print(text_message) 
                        
               #Overcomining fb messsneger limits:
                else:
                    text_message=''

                    for index, row in to_add.iterrows():
                        text_message=text_message+row['name'] + ' - ' + row['sellingModeformat'] + ' - '  + row['sellingModepriceamount'] + '\n'+ 'https://allegro.pl/oferta/' + row['id']+ '\n'+ '\n'

                    print(text_message) 
                    client.send(Message(text=text_message), thread_id=client.uid)

                with MongoClient(mongo_connection_string) as client_mongo:
                    mydb     = client_mongo["allegro"]
                    auction_tbl = mydb["auctions"]
                    rows_json=to_add.to_dict(orient='records')
                    auction_tbl.insert_many(rows_json)
                    #print('added')

        #Po 10 pętlach odśwież token (3h ważny)
    if(i == 10):
        refreshed_token=post_refresh_token(client_id, client_secret, refresh_token) 
        i=0
        
        access_token  = refreshed_token['access_token']
        refresh_token = refreshed_token['refresh_token']
        headers['Authorization'] = "Bearer {}".format(access_token)
        #print("refreshed")
        
    i=i+1
    #print(i)
    time.sleep(600)

In [ ]:
i=1
deepers={}
deepers[i]=categoryAllegro(77925)
#deepers[i]['path']= deepers[i]['name']
while True:
    deepers[i+1] = categoryAllegro(deepers[i]['parent']['id'])

    i=i+1
    if deepers[i]['parent'] is None:
        break
lenght_deepers=len(deepers)
deepers[lenght_deepers]['path']= deepers[lenght_deepers]['name']

for i in range(lenght_deepers-1,0,-1):
    deepers[i]['path']= deepers[i]['name']+ '/' + deepers[i+1]['path']

return deepers

In [ ]:
category_table.path[category_table['id']=='146704'].values[0]
rows_json[1]['categoryid']

In [ ]:
with MongoClient(mongo_connection_string) as client_mongo:
    mydb= client_mongo["allegro"]
    auction_tbl = mydb["auctions"]
    auction_list=list(auction_tbl.find({}))
         

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
json_normalize(auction_list)

In [ ]:
search_result=searchAllegro('honda accord IV')
search_result

table = json_normalize(search_result['items']['regular'])
table = table.rename(columns=lambda x: re.sub('\\.','',x))
current_time = dt.datetime.now()
table['keyword'] = k
table['record_creation_date']=current_time.strftime('%Y-%m-%d %H:%M:%S')
table

In [ ]:
table = json_normalize(search_result['items']['promoted'])
table

In [ ]:
for index, row in to_add.iterrows():
    text_message=row['name'] + ' - ' + row['sellingModeformat'] + ' - '  + row['sellingModepriceamount'] + '\n'+ 'https://allegro.pl/oferta/' + row['id']
    client.send(Message(text=text_message), thread_id=client.uid)
    #client.send(Message(text=text_message), thread_id=client.uid)
    time.sleep(10)

In [ ]:
for index, row in to_add.iterrows():
    text=row['name'] + ' - ' + row['sellingModeformat'] + ' - '  + row['sellingModepriceamount'] + '\n'+ 'https://allegro.pl/oferta/' + row['id']
    print(text)

In [ ]:
msg ="I'm dead!"
client.send(Message(text=msg), thread_id=client.uid)

In [ ]:
rows_json=categoryAggAllegro(c)
mycol.insert_many(rows_json)

In [ ]:
deeper = categoryAllegro(deeper['parent']['id'])
print(deeper)

In [ ]:
lenght_deepers=len(deepers)
deepers[lenght_deepers]['path']= deepers[lenght_deepers]['name']

for i in range(lenght_deepers-1,0,-1):
    deepers[i]['path']= deepers[i]['name']+ '/' + deepers[i+1]['path']
    print(i)

In [ ]:
for c in categories_to_add:
    with MongoClient(mongo_connection_string) as client_mongo:
        mydb     = client_mongo["allegro"]
        mycol    = mydb["categories"]
        #rows_json=categoryAllegro(c)
        mycol.insert_many(deepers)

https://stackoverflow.com/questions/4421207/how-to-get-the-last-n-records-in-mongodb

https://stackoverflow.com/questions/4421207/how-to-get-the-last-n-records-in-mongodb

https://cloud.mongodb.com/v2/5f7f7349ac5c6d14a6c43351#metrics/replicaSet/5f7f76ad04d74b62e1bc5aa4/explorer/allegro/searches/find